# overview

We start from the raw PDBbind dataset downloaded from http://www.pdbbind.org.cn/download.php

1. filter out those unable to process using RDKit.

2. Process the protein by only preserving the chains that with at least one atom within 10Å from any atom of the ligand.

3. Use p2rank to segment protein into blocks.

4. extract protein and ligand features.

5. construct the training and test dataset.


In [60]:
import time
from datetime import datetime as dt
def tt():
    print(time.strftime("%m-%d %H:%M:%S", time.localtime(time.time()+8*3600)))

In [61]:
CTN = True
tt()

10-10 17:37:23


In [62]:
tankbind_src_folder_path = "../tankbind/"
import sys
import torch
sys.path.insert(0, tankbind_src_folder_path)
tt()

10-10 17:37:24


In [63]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
tt()

10-10 17:37:24


# process the raw PDBbind dataset.

In [64]:
from utils import read_pdbbind_data
tt()

10-10 17:37:24


In [65]:
# raw PDBbind dataset could be downloaded from http://www.pdbbind.org.cn/download.php
pre = "/home/jovyan/data/pdbbind2020"
df_pdb_id = pd.read_csv(f'{pre}/index/INDEX_general_PL_name.2020', sep="  ", comment='#', header=None, names=['pdb', 'year', 'uid', 'd', 'e','f','g','h','i','j','k','l','m','n','o'], engine='python')
df_pdb_id = df_pdb_id[['pdb','uid']]
data = read_pdbbind_data(f'{pre}/index/INDEX_general_PL_data.2020')
data = data.merge(df_pdb_id, on=['pdb'])
tt()


10-10 17:37:24


In [66]:
ctn_path = "/home/jovyan/dataspace/NFT/main/CTNs"
tt()

10-10 17:37:24


# ligand file should be readable by RDKit.

In [67]:
from feature_utils import read_mol
tt()

10-10 17:37:24


In [68]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
pdb_list = []
probem_list = []
if not CTN:
    for pdb in tqdm(data.pdb):
        sdf_fileName = f"{pre}/pdbbind_files/{pdb}/{pdb}_ligand.sdf"
        mol2_fileName = f"{pre}/pdbbind_files/{pdb}/{pdb}_ligand.mol2"
        mol, problem = read_mol(sdf_fileName, mol2_fileName)
        if problem:
            probem_list.append(pdb)
            continue
        pdb_list.append(pdb)
    torch.save(pdb_list, f"{ctn_path}/pdb_list.pt")
pdb_list = torch.load(f"{ctn_path}/pdb_list.pt")
tt()

10-10 17:37:25


In [69]:
data = data.query("pdb in @pdb_list").reset_index(drop=True)
print(data.shape)
tt()

(19127, 7)
10-10 17:37:25


In [70]:
data.shape

(19127, 7)

### for ease of RMSD evaluation later, we renumber the atom index to be consistent with the smiles

In [71]:
from feature_utils import write_renumbered_sdf
pre_main = "/home/jovyan/dataspace/NFT/main"
toFolder = f"{pre_main}/renumber_atom_index_same_as_smiles"
os.system(f"mkdir -p {toFolder}")
# for PDB_Code in lig_id_dict.keys(), the sdf in NCI dataset will be used.
lig_id_dict = torch.load("../../dataspace/NFT/main/ligand_name_dicts/ligdict_id.pt")
tt()

10-10 17:37:25


In [72]:
if not CTN:
    for pdb in tqdm(pdb_list):
        if pdb not in lig_id_dict:  # Use PDBBind dataset sdf
            sdf_fileName = f"{pre}/pdbbind_files/{pdb}/{pdb}_ligand.sdf"
            mol2_fileName = f"{pre}/pdbbind_files/{pdb}/{pdb}_ligand.mol2"
            toFile = f"{toFolder}/{pdb}.sdf"
            write_renumbered_sdf(toFile, sdf_fileName, mol2_fileName)
        else:  # Use NCI dataset sdf
            sdf_fileName = f"/home/jovyan/dataspace/pdb_bind_2020/{pdb}/{pdb}_ligand.sdf"
            toFile = f"{toFolder}/{pdb}.sdf"
            write_renumbered_sdf(toFile, sdf_fileName, None)
tt()

10-10 17:37:25


In [73]:
from feature_utils import write_renumbered_sdf
pre_main = "/home/jovyan/dataspace/NFT/main"
toFolder = f"{pre_main}/renumber_atom_index_same_as_smiles"
os.system(f"mkdir -p {toFolder}")
tt()

10-10 17:37:26


In [74]:
if not CTN:
    
    for pdb in tqdm(pdb_list):
        sdf_fileName = f"{pre}/pdbbind_files/{pdb}/{pdb}_ligand.sdf"
        mol2_fileName = f"{pre}/pdbbind_files/{pdb}/{pdb}_ligand.mol2"
        toFile = f"{toFolder}/{pdb}.sdf"
        write_renumbered_sdf(toFile, sdf_fileName, mol2_fileName)


In [75]:
from feature_utils import write_renumbered_sdf
pre_main = "/home/jovyan/dataspace/NFT/main"
toFolder = f"{pre_main}/renumber_atom_index_same_as_smiles_from_NCI_sdf"
os.system(f"mkdir -p {toFolder}")

0

In [76]:
if False:
    import shutil
    os.system("mkdir -p /home/jovyan/dataspace/Predownloads/WZCase2")
    os.system("mkdir -p /home/jovyan/dataspace/Predownloads/PDBCase2")
    for pdb in lig_id_dict.keys():
        shutil.copy(f"/home/jovyan/dataspace/pdb_bind_2020/{pdb}/{pdb}_ligand.sdf", f"/home/jovyan/dataspace/Predownloads/WZCase2/{pdb}_NCI.sdf")
        shutil.copy(f"{pre}/pdbbind_files/{pdb}/{pdb}_ligand.sdf", f"/home/jovyan/dataspace/Predownloads/PDBCase2/{pdb}_PDBBind.sdf")

# process PDBbind proteins, removing extra chains, cutoff 10A

In [77]:
toFolder = f"{pre_main}/protein_remove_extra_chains_10A/"
os.system(f"mkdir -p {toFolder}")
tt()

10-10 17:37:26


In [78]:
input_ = []
cutoff = 10
for pdb in data.pdb.values:
    pdbFile = f"{pre}/pdbbind_files/{pdb}/{pdb}_protein.pdb"
    ligandFile = f"{pre_main}/renumber_atom_index_same_as_smiles/{pdb}.sdf"
    toFile = f"{toFolder}/{pdb}_protein.pdb"
    x = (pdbFile, ligandFile, cutoff, toFile)
    input_.append(x)
tt()

10-10 17:37:26


In [79]:
from feature_utils import select_chain_within_cutoff_to_ligand_v2
tt()

10-10 17:37:27


In [80]:
if not CTN:
    import mlcrate as mlc
    import os
    pool = mlc.SuperPool(64)
    pool.pool.restart()
    _ = pool.map(select_chain_within_cutoff_to_ligand_v2,input_)
    pool.exit()
tt()

10-10 17:37:27


In [81]:
# previously, I found that 2r1w has no chain near the ligand.
data = data.query("pdb != '2r1w'").reset_index(drop=True)
tt()

10-10 17:37:27


# p2rank segmentation

In [82]:
if not CTN:
    p2rank_prediction_folder = f"{pre_main}/p2rank_protein_remove_extra_chains_10A"
    os.system(f"mkdir -p {p2rank_prediction_folder}")
    ds = f"{p2rank_prediction_folder}/protein_list.ds"
    with open(ds, "w") as out:
        for pdb in data.pdb.values:
            out.write(f"../protein_remove_extra_chains_10A/{pdb}_protein.pdb\n")
tt()

10-10 17:37:27


In [83]:
if not CTN:
    # takes about 30 minutes.
    p2rank = "bash /home/jovyan/p2rank_2.3/prank"
    cmd = f"{p2rank} predict {ds} -o {p2rank_prediction_folder}/p2rank -threads 16"
    os.system(cmd)
tt()

10-10 17:37:27


In [84]:
if not CTN:
    data.to_csv(f"{pre_main}/data.csv")
tt()

10-10 17:37:28


### Continue From Here

In [85]:
data = pd.read_csv(f"{pre_main}/data.csv")
print(data.shape)
tt()

(19126, 8)
10-10 17:37:28


In [86]:
pdb_list = data.pdb.values
tt()

10-10 17:37:28


In [87]:
tankbind_data_path = f"{pre_main}/tankbind_data"
os.system(f"mkdir -p {tankbind_data_path}")

0

In [88]:
name_list = pdb_list
d_list = []

if not CTN:
    for name in tqdm(name_list):
        p2rankFile = f"{pre_main}/p2rank_protein_remove_extra_chains_10A/p2rank/{name}_protein.pdb_predictions.csv"
        d = pd.read_csv(p2rankFile)
        d.columns = d.columns.str.strip()
        d_list.append(d.assign(name=name))
    d = pd.concat(d_list).reset_index(drop=True)
    d.reset_index(drop=True).to_feather(f"{tankbind_data_path}/p2rank_result.feather")

In [89]:
d = pd.read_feather(f"{tankbind_data_path}/p2rank_result.feather")
tt()

10-10 17:37:29


In [90]:
pockets_dict = {}
if not CTN:
    for name in tqdm(name_list):
        pockets_dict[name] = d[d.name == name].reset_index(drop=True)
        torch.save(pockets_dict, f"{tankbind_data_path}/pocket_dict.pt")
pockets_dict = torch.load(f"{tankbind_data_path}/pocket_dict.pt")
tt()

10-10 17:37:31


# protein feature

In [91]:
from feature_utils import get_protein_feature
from feature_utils import nciyes_get_protein_feature
tt()

10-10 17:37:31


In [92]:
input_ = []
protein_embedding_folder = f"{tankbind_data_path}/gvp_protein_embedding"
os.system(f"mkdir -p {protein_embedding_folder}")
for pdb in pdb_list:
    proteinFile = f"{pre_main}/protein_remove_extra_chains_10A/{pdb}_protein.pdb"
    toFile = [f"{protein_embedding_folder}/{pdb}.pt", f"{protein_embedding_folder}/{pdb}_id.pt"]
    x = (pdb, proteinFile, toFile)
    input_.append(x)
tt()

10-10 17:37:35


In [93]:
def get_full_id(full_id_ls: list, resname):
    chain_id = full_id_ls[2]
    res_id = full_id_ls[3][1]
    return chain_id + "_" + str(res_id) + "_" + resname
tt()

10-10 17:37:36


In [94]:
from Bio.PDB import PDBParser
from feature_utils import get_clean_res_list
import torch
torch.set_num_threads(1)

def batch_run(x):
    protein_dict = {}
    protein_id_dict = {}
    pdb, proteinFile, toFile = x
    parser = PDBParser(QUIET=True)
    s = parser.get_structure(pdb, proteinFile)
    res_list = get_clean_res_list(s.get_residues(), verbose=False, ensure_ca_exist=True)
    res_full_id_list = [get_full_id(x.full_id, x.get_resname()) for x in res_list]
    protein_dict[pdb], protein_id_dict[pdb] = nciyes_get_protein_feature(res_list, res_full_id_list)
    torch.save(protein_dict, toFile[0])
    torch.save(protein_id_dict, toFile[1])
tt()

10-10 17:37:36


In [95]:
if not CTN:
    import mlcrate as mlc
    import os
    pool = mlc.SuperPool(64)
    pool.pool.restart()
    _ = pool.map(batch_run,input_)
    pool.exit()
tt()

10-10 17:37:36


In [96]:
import torch
protein_dict = {}
protein_id_dict = {}
for pdb in tqdm(pdb_list):
    protein_dict.update(torch.load(f"{protein_embedding_folder}/{pdb}.pt"))
    protein_id_dict.update(torch.load(f"{protein_embedding_folder}/{pdb}_id.pt"))
tt()

100%|██████████| 19126/19126 [05:10<00:00, 61.54it/s] 

10-10 17:42:47


# Compound Features

In [97]:
from feature_utils import extract_torchdrug_feature_from_mol
if not CTN:
    compound_dict = {}
    skip_pdb_list = []
    for pdb in tqdm(pdb_list):
        mol, _ = read_mol(f"{pre_main}/renumber_atom_index_same_as_smiles/{pdb}.sdf", None)
        # extract features from sdf.
        try:
            compound_dict[pdb] = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)  # self-dock set has_LAS_mask to true
        except Exception as e:
            print(e)
            skip_pdb_list.append(pdb)
            print(pdb)
    torch.save(compound_dict, f"{tankbind_data_path}/compound_torchdrug_features.pt")
tt()

10-10 17:42:47


In [98]:
if not CTN:
    torch.save(compound_dict, f"{tankbind_data_path}/compound_torchdrug_features.pt")

In [99]:
compound_dict = torch.load(f"{tankbind_data_path}/compound_torchdrug_features.pt")
skip_pdb_list = ['3kqs']
tt()

10-10 17:43:20


In [100]:
skip_pdb_list = ['3kqs']
tt()

10-10 17:43:20


In [101]:
data = data.query("pdb not in @skip_pdb_list").reset_index(drop=True)
tt()

10-10 17:43:20


### NCI Features

In [102]:
lig_nameid_dict = torch.load("../../dataspace/NFT/main/ligand_name_dicts/ligdict_nameid.pt")
lig_id_dict = torch.load("../../dataspace/NFT/main/ligand_name_dicts/ligdict_id.pt")
NCI_df = pd.read_csv("/home/jovyan/dataspace/processed_nci_data/v10.8/Data.v10.8.NCIs.csv")
tt()

10-10 17:43:22


In [ ]:
pdb_list

In [118]:
# for pdb in tqdm(pdb_list):
from collections import defaultdict
from tqdm import tqdm
if CTN:
    nci_info = {}
    nci_save_path = "/home/jovyan/dataspace/NFT/main/nci_protein_ligand_matrix"
    # os.system(f"rm -rf {nci_save_path}")
    os.system(f"mkdir -p {nci_save_path}")
    for pdb in tqdm(pdb_list):
        #print(pdb)
        if True:
            if (pdb not in lig_nameid_dict) and (pdb not in lig_id_dict):
                nci_info[pdb] = "BAD_NoLigName"


            # Lig
            mol, _ = read_mol(f"{pre_main}/renumber_atom_index_same_as_smiles/{pdb}.sdf", None)
            
            # Protein
            proteinFile = f"{pre_main}/protein_remove_extra_chains_10A/{pdb}_protein.pdb"
            toFile = [f"{protein_embedding_folder}/{pdb}.pt", f"{protein_embedding_folder}/{pdb}_id.pt"]
            ebd = torch.load(toFile[0])[pdb]
            protein_ids = torch.load(toFile[1])[pdb]
            
            # inverse protein_ids:
            protein_id_inverse = defaultdict(list)
            for _id, _name in protein_ids.items():
                protein_id_inverse[_name[0]].append(_id)
            
            protein_id_inverse = {k:v for k, v in protein_id_inverse.items()}
            
            # inverse lig_ids:
            
            if pdb in lig_nameid_dict:
                liglen = len(lig_nameid_dict[pdb])
                lig_id_inverse = {_value[1]:_key for _key, _value in lig_nameid_dict[pdb].items()}
            elif pdb in lig_id_dict:
                liglen = len(lig_id_dict[pdb])
                lig_id_inverse = {_value:_key for _key, _value in lig_id_dict[pdb].items()}
                

            
            pl_0 = ebd[0].shape[0]
            pl_1 = len(protein_ids)
            ll_0 = mol.GetNumAtoms()
            ll_1 = liglen

            nci_matrix = torch.zeros(pl_0, ll_0)

            if pl_0 != pl_1:
                nci_info[pdb] = "BAD_ResNum"
                torch.save({pdb:nci_matrix_0}, f"{nci_save_path}/{pdb}_nci_matrix.pt")
                continue
            elif ll_0 != ll_1:
                nci_info[pdb] = "BAD_AtomNum"
                torch.save({pdb:nci_matrix_0}, f"{nci_save_path}/{pdb}_nci_matrix.pt")
                continue

            ncis = NCI_df[NCI_df.PDB_Code==pdb]
            if len(ncis) == 0:
                nci_info[pdb] = "BAD_EmptyNCI"

            nci_matrix = torch.zeros(pl_0, ll_0)
            nci_matrix_0 = torch.zeros(pl_0, ll_0)
            
            if pdb in nci_info:
                torch.save({pdb:nci_matrix_0}, f"{nci_save_path}/{pdb}_nci_matrix.pt")
                continue
            
            for line in ncis.iterrows():
                ResFullID, l_id = line[1]['ResFullID'], line[1]['l_id']
                
                try:
                    p_index = protein_id_inverse[ResFullID]
                except:
                    nci_info[pdb] = "BAD_ResKey"
                    torch.save({pdb:nci_matrix_0}, f"{nci_save_path}/{pdb}_nci_matrix.pt")
                    break

                try:
                    l_index = lig_id_inverse[l_id]
                except:
                    nci_info[pdb] = "BAD_AtomKey"
                    torch.save({pdb:nci_matrix_0}, f"{nci_save_path}/{pdb}_nci_matrix.pt")
                    break
                
                if len(p_index) > 1:
                    nci_info[pdb] = "BAD_ResIncoherant"
                    torch.save({pdb:nci_matrix_0}, f"{nci_save_path}/{pdb}_nci_matrix.pt")
                    break
                else:
                    nci_matrix[p_index[0]][l_index] = 1
                    
            if pdb in nci_info:
                continue
            else:
                nci_info[pdb] = "YES"
                torch.save({pdb:nci_matrix}, f"{nci_save_path}/{pdb}_nci_matrix.pt")
    torch.save(nci_info, "/home/jovyan/dataspace/NFT/main/nci_info.pt")
tt()

100%|██████████| 19126/19126 [11:27<00:00, 27.83it/s]

10-10 18:07:36


In [104]:
nci_info = torch.load("/home/jovyan/dataspace/NFT/main/nci_info.pt")
for k, v in nci_info.items():
    if v == "BAD_ResNum" or v == "BAD_AtomNum":
        print(k)
tt()

1w8l
3fqa
1zsb
1wkm
2w97
2b1r
2d2v
5eb2
4fci
4fck
1a0t
1b74
5oxm
2igw
2vl1
1qpb
6k2n
1o4l
1o4m
1o4n
3eor
1ozv
1tnk
4bh4
5ele
1iig
2b1q
1w8m
2o9r
2sfp
3oay
3ern
3vzg
3zst
4bgx
3f39
2zgm
1ga8
1af6
1bra
1d5r
1tnl
2ay2
1f2p
1f2o
3ehn
4bh3
1tnj
5oxn
1wdq
1gbt
2bmv
3pce
4y4g
6evq
2ks9
1jfh
1qq9
5tyh
1p2g
1uxb
4b0j
2ay9
4tim
6ghj
5oxk
6g47
1w1v
1m0n
2aac
5fsb
2igv
2ay8
1ukt
2ay1
1jdj
1n1g
1o4g
1uxa
2ay7
2hdu
3pcg
1m0o
5elf
1qi0
1o4j
5fsc
4jck
6rnt
3pcb
4u5l
4zsm
1o4r
6mli
1o4h
1t4s
1gbq
1o4o
2aez
4gne
1o4q
6c3n
1wdr
3ad7
3b3s
1gwv
2ay5
2jst
4o3a
4o3c
4wef
5tpb
1o4p
2c4v
6mia
1cze
2kfh
4lnf
4lno
1o4i
4q7w
6fhu
3rxb
5eld
1o8b
5aqj
5aqp
3elc
5k5s
1gym
1lt5
2krd
3bra
3gsg
4okp
1pfu
1dg9
2qwb
6a9c
5oyd
1ny2
4a4h
6er4
2bge
4yw2
5fov
4oyt
1czc
1p17
4z46
6cw4
6evm
6mj7
4yz5
2a29
4ym2
6jjm
4pd5
1e34
4a4f
1iih
2k2r
2wtx
3g31
4b35
4cd8
4u5u
1ms0
2ay4
2pri
1eef
1o4k
2kfg
3alt
4x14
1tng
4zv2
2vmc
1f4e
2kdh
4bgy
4k3l
2rnw
3ckb
4a4g
1hqg
1ux7
1w7h
1wbg
1wbo
2r0y
3ap7
3d9o
3ob0
4fe9
4k0o
4k64
4k72
4lkh
5tpc


In [105]:
if not CTN:
    torch.save(nci_info, "/home/jovyan/dataspace/NFT/main/nci_info.pt")

nci_info = torch.load("/home/jovyan/dataspace/NFT/main/nci_info.pt")
tt()

10-10 17:51:44


In [106]:
nci_info_stat = defaultdict(int)
for k,v in nci_info.items():
    nci_info_stat[v] += 1
print(nci_info_stat)
tt()

defaultdict(<class 'int'>, {'YES': 10727, 'BAD_AtomNum': 5561, 'BAD_EmptyNCI': 2680, 'BAD_NoLigName': 36, 'BAD_ResKey': 119, 'BAD_ResIncoherant': 3})
10-10 17:51:44


In [107]:
nci_dict = {}
if not CTN:
    for pdb in tqdm(pdb_list):
        if pdb not in nci_info or nci_info[pdb] != "YES":
            continue
        else:
            nci_dict.update(torch.load(f"{nci_save_path}/{pdb}_nci_matrix.pt"))
    torch.save(nci_dict, f"{tankbind_data_path}/nci_matrix.pt")
tt()

10-10 17:51:44


In [108]:
nci_dict = torch.load(f"{tankbind_data_path}/nci_matrix.pt")
tt()

10-10 17:51:49


# construct dataset.

In [109]:
# we use the time-split defined in EquiBind paper.
# https://github.com/HannesStark/EquiBind/tree/main/data
valid = np.loadtxt("/home/jovyan/data/Equiband/timesplit_no_lig_overlap_val", dtype=str)
test = np.loadtxt("/home/jovyan/data/Equiband/timesplit_test", dtype=str)
def assign_group(pdb, valid=valid, test=test):
    if pdb in valid:
        return 'valid'
    if pdb in test:
        return 'test'
    return 'train'

data['group'] = data.pdb.map(assign_group)
tt()

10-10 17:51:49


In [110]:
print(data.value_counts("group"))
tt()

group
train    17794
valid      968
test       363
dtype: int64
10-10 17:51:49


In [111]:
data['name'] = data['pdb']
tt()

10-10 17:51:49


In [112]:
if not CTN:
    info = []
    for i, line in tqdm(data.iterrows(), total=data.shape[0]):
        pdb = line['pdb']
        uid = line['uid']
        # smiles = line['smiles']
        smiles = ""
        affinity = line['affinity']
        group = line['group']

        compound_name = line['name']
        protein_name = line['name']

        pocket = pockets_dict[pdb].head(10)
        pocket.columns = pocket.columns.str.strip()
        pocket_coms = pocket[['center_x', 'center_y', 'center_z']].values
        
        has_nci = (nci_info[protein_name] == "YES")
        compound_sdf_from_nci = (protein_name in lig_id_dict.keys())
        # native block.
        info.append([protein_name, compound_name, pdb, smiles, affinity, uid, None, True, False, group, has_nci, compound_sdf_from_nci])
        # protein center as a block.
        protein_com = protein_dict[protein_name][0].numpy().mean(axis=0).astype(float).reshape(1, 3)
        info.append([protein_name, compound_name, pdb+"_c", smiles, affinity, uid, protein_com, False, False, group, has_nci, compound_sdf_from_nci])
        
        for idx, pocket_line in pocket.iterrows():
            pdb_idx = f"{pdb}_{idx}"
            info.append([protein_name, compound_name, pdb_idx, smiles, affinity, uid, pocket_coms[idx].reshape(1, 3), False, False, group, has_nci, compound_sdf_from_nci])
    info = pd.DataFrame(info, columns=['protein_name', 'compound_name', 'pdb', 'smiles', 'affinity', 'uid', 'pocket_com', 
                                    'use_compound_com', 'use_whole_protein',
                                    'group', 'has_nci_info', 'compound_sdf_from_nci'])
    torch.save(info, f"{tankbind_data_path}info.pt")
info = torch.load(f"{tankbind_data_path}info.pt")
tt()



10-10 17:51:50


In [113]:
info.shape
tt()

10-10 17:51:51


In [114]:
info.head(5)

,protein_name,compound_name,pdb,smiles,affinity,uid,pocket_com,use_compound_com,use_whole_protein,group,has_nci_info,compound_sdf_from_nci
0,3zzf,3zzf,3zzf,,0.4,Q01217,None,True,False,train,True,False
1,3zzf,3zzf,3zzf_c,,0.4,Q01217,"[[5.51331901550293, 36.50146484375, 14.4291219...",False,False,train,True,False
2,3zzf,3zzf,3zzf_0,,0.4,Q01217,"[[9.2232, 36.6453, 4.2458]]",False,False,train,True,False
3,3zzf,3zzf,3zzf_1,,0.4,Q01217,"[[-3.9652, 36.9019, 2.8611]]",False,False,train,True,False
4,3zzf,3zzf,3zzf_2,,0.4,Q01217,"[[16.5628, 39.1406, 26.3637]]",False,False,train,True,False


In [115]:

from data_nci import NFTankBindDataSet
tt()

10-10 17:51:51


In [116]:
toFilePre = f"{tankbind_data_path}/dataset"
if CTN:
    os.system(f"rm -rf {toFilePre}")
    os.system(f"mkdir -p {toFilePre}")
    dataset = NFTankBindDataSet(toFilePre, data=info, protein_dict=protein_dict, compound_dict=compound_dict, nci_dict=nci_dict)

Processing...
Done!


['/home/jovyan/dataspace/NFT/main/tankbind_data/dataset/processed/data.pt', '/home/jovyan/dataspace/NFT/main/tankbind_data/dataset/processed/protein.pt', '/home/jovyan/dataspace/NFT/main/tankbind_data/dataset/processed/compound.pt', '/home/jovyan/dataspace/NFT/main/tankbind_data/dataset/processed/nci.pt']


KeyboardInterrupt: 

In [ ]:
dataset[0]

In [ ]:
dataset = NFTankBindDataSet(toFilePre)
tt()

['/home/jovyan/dataspace/NFT/main/tankbind_data/dataset/processed/data.pt', '/home/jovyan/dataspace/NFT/main/tankbind_data/dataset/processed/protein.pt', '/home/jovyan/dataspace/NFT/main/tankbind_data/dataset/processed/compound.pt', '/home/jovyan/dataspace/NFT/main/tankbind_data/dataset/processed/nci.pt']


KeyboardInterrupt: 

In [ ]:
t = []
data = dataset.data
pre_pdb = None
for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    pdb = line['compound_name']
    d = dataset[i]
    p_length = d['node_xyz'].shape[0]
    c_length = d['coords'].shape[0]
    y_length = d['y'].shape[0]
    num_contact = (d.y > 0).sum()
    t.append([i, pdb, p_length, c_length, y_length, num_contact])
tt()


  0%|          | 22/162029 [00:00<15:42, 171.89it/s]


KeyError: '1w8l'

In [ ]:
# data = data.drop(['p_length', 'c_length', 'y_length', 'num_contact'], axis=1)

In [ ]:
t = pd.DataFrame(t, columns=['index', 'pdb' ,'p_length', 'c_length', 'y_length', 'num_contact'])
t['num_contact'] = t['num_contact'].apply(lambda x: x.item())

In [ ]:
data = pd.concat([data, t[['p_length', 'c_length', 'y_length', 'num_contact']]], axis=1)

In [ ]:
native_num_contact = data.query("use_compound_com").set_index("protein_name")['num_contact'].to_dict()
data['native_num_contact'] = data.protein_name.map(native_num_contact)
# data['fract_of_native_contact'] = data['num_contact'] / data['native_num_contact']

In [ ]:
torch.save(data, f"{toFilePre}/processed/data.pt")

In [ ]:
info = torch.load(f"{toFilePre}/processed/data.pt")


In [ ]:
test = info.query("group == 'test'").reset_index(drop=True)
test_pdb_list = info.query("group == 'test'").protein_name.unique()

In [ ]:
subset_protein_dict = {}
for pdb in tqdm(test_pdb_list):
    subset_protein_dict[pdb] = protein_dict[pdb]

100%|██████████| 363/363 [00:00<00:00, 251866.39it/s]


In [ ]:
subset_compound_dict = {}
for pdb in tqdm(test_pdb_list):
    subset_compound_dict[pdb] = compound_dict[pdb]

100%|██████████| 363/363 [00:00<00:00, 182208.28it/s]


In [ ]:

toFilePre = f"{pre}/test_dataset"
os.system(f"mkdir -p {toFilePre}")
dataset = NFTankBindDataSet(toFilePre, data=test, protein_dict=subset_protein_dict, compound_dict=subset_compound_dict)

In [ ]:
def canonical_smiles(smiles):
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))